In [5]:

from function_filePrep import tickerList, download_data,missing_days_andIndexTimeZone, to_month_and_add_monthYear_columns
import yfinance
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error,mean_squared_error
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from dateutil.relativedelta import relativedelta
# MAPE
def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


df_ticker_price = download_data(list_stocks=tickerList,start_date = '2016-01-01', end_date = '2024-09-30')
df_ticker_price1=missing_days_andIndexTimeZone(df_ticker_price)
monthly_data=to_month_and_add_monthYear_columns(df_ticker_price1)

[*********************100%***********************]  30 of 30 completed


In [6]:

#### CREATING FUNCTION TO CREATE THAT DATAFRAME FROM THE TOP 
cutoff_points = ['2023-09-30','2023-03-31','2022-09-30','2022-03-31']
def Error_Rate_Horizon(stockToSelect: str, cutoff_points: list[str] = cutoff_points, horizon:int=12 ):    
    stock_hist = monthly_data[[stockToSelect,'month','year']].dropna().copy()
    
    df_allEntries = pd.DataFrame({
    'Date': [],
    stockToSelect: [],
    'Pred': [],
    'CutoffDate': []
    })

        
    def pred_and_error(stock,cutoff_date,months_forecast):
        date_split = pd.to_datetime(cutoff_date)
        nextMonth = date_split + relativedelta(months=1)
        month12 = date_split + relativedelta(months=months_forecast)
        #months_to_forecast = 12 
        stock_train = stock_hist[stock][:date_split]
        stock_test = stock_hist[stock][nextMonth:month12]
        # forecasting 12 months if we can forecast 12 months, othewise the max allowed to forecast
        model = ExponentialSmoothing(stock_train, trend='mul', seasonal='add', seasonal_periods=12)
        fitted_model = model.fit()
        forecast = fitted_model.forecast(steps=months_forecast) 
        
        mini_df=stock_test.reset_index().merge(forecast.reset_index().rename(columns={'index':'Date',0:'Pred'}), on='Date')
        #df_allEntries=pd.concat([df_allEntries,mini_df])
        mini_df['CutoffDate'] = cutoff_date
        return mini_df

    for cutoff_point in cutoff_points: 
        df_allEntries=pd.concat([df_allEntries,pred_and_error(stockToSelect,cutoff_point,horizon)])

    df_allEntries['monthDiff']=df_allEntries.apply(lambda x: relativedelta( x['Date'], pd.to_datetime(x['CutoffDate'])).years *12 + relativedelta( x['Date'], pd.to_datetime(x['CutoffDate'])).months ,axis=1)


    MAPE_calc = df_allEntries.groupby('monthDiff').apply(lambda x: mean_absolute_percentage_error(x[stockToSelect],x['Pred']))

    mse_calc=df_allEntries.groupby('monthDiff').apply(lambda x: np.sqrt( mean_squared_error(x[stockToSelect],x['Pred']) ))

    mae_calc=df_allEntries.groupby('monthDiff').apply(lambda x: mean_absolute_error(x[stockToSelect],x['Pred']) )

    return MAPE_calc.reset_index().rename(columns={0:'MAPE'}).merge(mse_calc.reset_index().rename(columns={0:'MSE'}), on='monthDiff').merge(mae_calc.reset_index().rename(columns={0:'MAE'}),on='monthDiff')

#%%
cutoff_points = ['2023-09-30','2023-03-31','2022-09-30','2022-03-31']


,monthDiff,MAPE,MSE,MAE
0,1,9.409898,10.828900,8.936466
1,2,6.696082,8.926781,7.219778
2,3,7.215105,8.745307,7.873639
3,4,12.315918,18.112126,12.489422
4,5,12.961112,16.512841,14.758502
5,6,15.402781,19.926055,16.087545
6,7,28.028287,30.672108,25.416215
7,8,24.933307,28.158219,24.676792
8,9,11.064490,17.736885,11.908552
9,10,19.031763,20.876570,18.468225


In [16]:
Error_Rate_Horizon('KO')

,monthDiff,MAPE,MSE,MAE
0,1,6.485514,3.731938,3.624725
1,2,5.546662,3.354291,3.224412
2,3,1.517711,1.015997,0.891344
3,4,2.550275,1.776613,1.504831
4,5,2.362094,1.726170,1.394519
5,6,3.479714,2.479725,1.967307
6,7,4.839708,3.374914,2.620568
7,8,3.999165,2.642484,2.257615
8,9,3.886896,2.581611,2.275744
9,10,4.236198,2.693233,2.512889
